<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/char-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
import keras.utils

sh_url = 'https://homl.info/shakespeare'
sh_file = "sh.txt"
filepath = keras.utils.get_file(sh_file, sh_url)

Using TensorFlow backend.


1122304/1115394 [==============================] - 0s 0us/step


In [0]:
with open(filepath) as f:
  sh_txt = f.read()  

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([sh_txt])

In [5]:
print(tokenizer.texts_to_sequences(['Hello world']))
print(tokenizer.sequences_to_texts([[7,2,12,12,4], [1,17,4,9,12,13]]))

[[7, 2, 12, 12, 4, 1, 17, 4, 9, 12, 13]]
['h e l l o', '  w o r l d']


In [6]:
max_id = len(tokenizer.word_index)
print(max_id)

39


In [7]:
print(tokenizer.word_index)

{' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'i': 6, 'h': 7, 's': 8, 'r': 9, 'n': 10, '\n': 11, 'l': 12, 'd': 13, 'u': 14, 'm': 15, 'y': 16, 'w': 17, ',': 18, 'c': 19, 'f': 20, 'g': 21, 'b': 22, 'p': 23, ':': 24, 'k': 25, 'v': 26, '.': 27, "'": 28, ';': 29, '?': 30, '!': 31, '-': 32, 'j': 33, 'q': 34, 'x': 35, 'z': 36, '3': 37, '&': 38, '$': 39}


In [0]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([sh_txt]))

In [9]:
print(encoded[10:20])

[36  2 10 24 11 22  2 20  4  9]


In [10]:
print(len(encoded))

1115394


In [11]:
dataset_size = len(encoded)
train_size = dataset_size * 90 // 100
val_size = dataset_size - train_size
X_train = encoded[:train_size]
X_val = encoded[:val_size]
print(len(X_train), len(X_val))

1003854 111540


In [0]:
n_steps = 100  # number of chars in a sequence
window_length = n_steps + 1  # target is the char just after n_steps.

In [0]:
def get_chunks(sequence, n_steps):
  # 'abcdefghijkl'  when n_steps = 2 should return ['ab','bc','cd','ef',....'jk'] and ['c','d',....'l']
  index = 0
  X = list()
  y = list()
  while index + n_steps <= len(sequence)-1:
    print(index, index+ n_steps, len(sequence)-1)
    X.append(sequence[index:index+n_steps])
    y.append(sequence[index+n_steps])
    index += 1
  return X,y

In [24]:
X, y = get_chunks('abcdefghijkl', 2)
print(X, y)

0 2 11
1 3 11
2 4 11
3 5 11
4 6 11
5 7 11
6 8 11
7 9 11
8 10 11
9 11 11
['ab', 'bc', 'cd', 'de', 'ef', 'fg', 'gh', 'hi', 'ij', 'jk'] ['c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']
